# Análise de preços de Kitnets na cidade de Brasília e entornos

#### Este projeto visa trazer informações importantes em relação aos aluguéis de kitnets em Brasília.

##### Para trazer essas informações utilizamos a técnica de Webscrapping do site www.dfimoveis.com.br . Por meio dessa técnica recolhemos as informações de todos os anúncios de kitnets nas cidades de brasília

### Importanto as bibliotecas
##### Vamos começar instalando a biblioteca selenium no nosso IDLE e após isso importando todas as bibliotecas necessárias para a preparação dos dados

In [1]:
!pip install selenium

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import sqlite3
from pprint import pprint
import warnings
warnings.filterwarnings("ignore")
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By

### Obtendo a Estrutura do Site

##### Após importar as bibliotecas, iremos definir a url em que será feito o Webscrapping e definir os usuários que serão utilizados para acessar o site

In [3]:
url = "https://www.dfimoveis.com.br/aluguel/df/todos/kitnet?pagina={npagina}"

userAgents=[
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/74.0.3729.157 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15"
]

##### Agora iremos acessar o site e olhar se conseguimos acessar o seu conteúdo com sucesso

In [4]:
doc = requests.get(url.format(npagina=1), headers={"User-agent": userAgents[1]})

In [5]:
olhadinha = 700
print(str(doc.content[:olhadinha]) + "...")                      

b'\r\n\r\n\r\n\r\n<!DOCTYPE html>\r\n<html lang="pt-BR">\r\n<head>\r\n        <script>\r\n            (function (w, d, s, l, i) {\r\n                w[l] = w[l] || []; w[l].push({\r\n                    \'gtm.start\':\r\n                        new Date().getTime(), event: \'gtm.js\'\r\n                }); var f = d.getElementsByTagName(s)[0],\r\n                    j = d.createElement(s), dl = l != \'dataLayer\' ? \'&l=\' + l : \'\'; j.async = true; j.src =\r\n                    \'https://www.googletagmanager.com/gtm.js?id=\' + i + dl; f.parentNode.insertBefore(j, f);\r\n            })(window, document, \'script\', \'dataLayer\', \'GTM-54QGRMD\');\r\n        </script>\r\n    <meta charset="utf-8">\r\n    <meta name="viewport" content="width=dev'...


##### Como se trata de um site escrito em HTML, é necessário transformar os dados em formato de DOM a fim de coletar as informações que procuramos através das tags da estrutura HTML do site

In [6]:
analisador = BeautifulSoup(doc.content, "html.parser")
analisador


<!DOCTYPE html>

<html lang="pt-BR">
<head>
<script>
            (function (w, d, s, l, i) {
                w[l] = w[l] || []; w[l].push({
                    'gtm.start':
                        new Date().getTime(), event: 'gtm.js'
                }); var f = d.getElementsByTagName(s)[0],
                    j = d.createElement(s), dl = l != 'dataLayer' ? '&l=' + l : ''; j.async = true; j.src =
                    'https://www.googletagmanager.com/gtm.js?id=' + i + dl; f.parentNode.insertBefore(j, f);
            })(window, document, 'script', 'dataLayer', 'GTM-54QGRMD');
        </script>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="ie=edge" http-equiv="X-UA-Compatible"/>
<title> Kitnets para alugar no DF    - DFimoveis.com</title>
<meta content="dfimoveis.com" name="author"/>
<meta content="Aluguel de 978 Kitnets no Distrito Federal - DF. Encontre Kitnets para alugar no Distrito Federal - DF." name="description"/>


##### Ao analisar o DOM do site do DF imovéis, percebe-se que cada imóvel é separado pela tag: li class="property-list__item". Por esse motivo iremos separar os imoveis por meio dessa tag utilizando o método "find_all" do BeautifulSoap

In [7]:
imoveis = analisador.find_all("li", class_="property-list__item")

##### Agora é possível ver quantos imóveis nós temos em cada página de busca

In [8]:
anuncios_por_pagina = len(imoveis)
anuncios_por_pagina

30

##### É possível também verificar os primeiros 200 caracteres de cada código dos imóveis a fim de verificar se está tudo certo

In [9]:
for i in imoveis[:5]:
  print(str(i).replace("   ","")[:200]+"...\n")

<li class="property-list__item" itemprop="containsPlace" itemscope="" itemtype="https://schema.org/SingleFamilyResidence">
<meta itemprop="name"/>
<meta content="SEPS 713/913 - ED. GOLDEN PLACE - EXCE...

<li class="property-list__item" itemprop="containsPlace" itemscope="" itemtype="https://schema.org/SingleFamilyResidence">
<meta itemprop="name"/>
<meta content="Granville Negócios Imobiliários Vende:...

<li class="property-list__item" itemprop="containsPlace" itemscope="" itemtype="https://schema.org/SingleFamilyResidence">
<meta itemprop="name"/>
<meta content="Studio-Enxoval Completo e Serv. De Hot...

<li class="property-list__item" itemprop="containsPlace" itemscope="" itemtype="https://schema.org/SingleFamilyResidence">
<meta itemprop="name"/>
<meta content="Granville Negócios Imobiliários Aluga:...

<li class="property-list__item" itemprop="containsPlace" itemscope="" itemtype="https://schema.org/SingleFamilyResidence">
<meta itemprop="name"/>
<meta content="CCSW 02, Ed. Linea

### Extraindo as informações

##### Após estruturar todos os dados, podemos agora procurar por tags com informações importantes para a nossa análise de dados como o ID do anúncio do imóvel, preço, endereço e informações adicionais de cada kitnet.

##### Dessa forma, vamos verificar, primeiramente cada uma das informações que queremos buscar

In [10]:
#ID do Imóvel
imoveis[0].find_next("div", class_="property__link hide-mobile gerenciar-favorito").get("data-id")

'361049'

In [11]:
#Título do Anúncio
imoveis[0].find("a", class_="gtm-imovel texto-uma-linha").text.strip()

#No título do imóvel já dá para saber o endereço, o bairro e a cidade devido ao padrão do anúncio do site

'SEPS 713/913, ASA SUL, BRASILIA'

In [12]:
#Preço e metro quadrado
imoveis[0].find("h4", class_="property__subtitle hide-mobile").text.strip()

'R$1.200\nm² R$42'

In [13]:
#Quartos, suítes e vaga
quartos_suite_vaga = []
for i in imoveis[1].find_all("li", class_=""):
    quartos_suite_vaga.append(str(i))
    
quartos_suite_vaga

['<li>1 Quarto</li>']

In [14]:
str(imoveis[1].find_all("li", class_=""))


'[<li>1 Quarto</li>]'

##### Com os dados que serão extraídos bem definidos, o próximo passo consiste em extrair a informação de cada a anúncio de todas as páginas do site. Pra começar, então, é necessário saber quantas páginas no total tem no site. Para isso, fizemos o cálculo da quantidade total de anúncios disponíveis dividido pela quantidade de anúncios por páginas.

In [15]:
string_total_anuncios = analisador.find("h1", itemprop="name").text.strip()

In [16]:
import re
import math

In [17]:
quantidade_de_anuncios = int(re.findall("\d+", string_total_anuncios)[0])
quantidade_de_anuncios

978

In [18]:
numero_de_paginas = math.ceil(quantidade_de_anuncios / anuncios_por_pagina)
numero_de_paginas

33

In [19]:
fichas = []

for pagina in range(1,numero_de_paginas+1):
    print('Estou aqui: ' + url.format(npagina=pagina))
    
    # faz a requisição da página e armazena o html
    doc = requests.get(url.format(npagina=pagina))
    # analisa o HTML e armazena em uma variável no formato DOM
    analisador = BeautifulSoup(doc.content, 'html.parser')
    # extrair somente a lista de imoveis de todo o html
    imoveis = analisador.find_all("li", class_="property-list__item")
    
    for unidade in imoveis:
        uni = {} 
        #id
        uni['id'] = unidade.find_next("div", class_="property__link hide-mobile gerenciar-favorito").get("data-id")
        #título
        uni['titulo'] = unidade.find("a", class_="gtm-imovel texto-uma-linha").text.strip()
        #preco e metro quadrado
        uni['preco_e_metro_qd'] = unidade.find("h4", class_="property__subtitle hide-mobile").text.strip()
        #quarto, suite e vaga
        try:
            uni['quarto_suite_vaga'] = str(unidade.find_all("li", class_=""))
        except:
            pass
        fichas.append(uni)

Estou aqui: https://www.dfimoveis.com.br/aluguel/df/todos/kitnet?pagina=1
Estou aqui: https://www.dfimoveis.com.br/aluguel/df/todos/kitnet?pagina=2
Estou aqui: https://www.dfimoveis.com.br/aluguel/df/todos/kitnet?pagina=3
Estou aqui: https://www.dfimoveis.com.br/aluguel/df/todos/kitnet?pagina=4
Estou aqui: https://www.dfimoveis.com.br/aluguel/df/todos/kitnet?pagina=5
Estou aqui: https://www.dfimoveis.com.br/aluguel/df/todos/kitnet?pagina=6
Estou aqui: https://www.dfimoveis.com.br/aluguel/df/todos/kitnet?pagina=7
Estou aqui: https://www.dfimoveis.com.br/aluguel/df/todos/kitnet?pagina=8
Estou aqui: https://www.dfimoveis.com.br/aluguel/df/todos/kitnet?pagina=9
Estou aqui: https://www.dfimoveis.com.br/aluguel/df/todos/kitnet?pagina=10
Estou aqui: https://www.dfimoveis.com.br/aluguel/df/todos/kitnet?pagina=11
Estou aqui: https://www.dfimoveis.com.br/aluguel/df/todos/kitnet?pagina=12
Estou aqui: https://www.dfimoveis.com.br/aluguel/df/todos/kitnet?pagina=13
Estou aqui: https://www.dfimoveis.

##### Vamos verificar o resultado do nosso scrapping e conferir os dados

In [20]:
fichas

[{'id': '361049',
  'titulo': 'SEPS 713/913, ASA SUL, BRASILIA',
  'preco_e_metro_qd': 'R$1.200\nm² R$42',
  'quarto_suite_vaga': '[<li>1 Quarto</li>, <li>1 Vaga</li>]'},
 {'id': '359071',
  'titulo': 'SCRN 708/709, ASA NORTE, BRASILIA',
  'preco_e_metro_qd': 'R$1.200\nm² R$40',
  'quarto_suite_vaga': '[<li>1 Quarto</li>]'},
 {'id': '429066',
  'titulo': 'CCSW 1, SUDOESTE, BRASILIA',
  'preco_e_metro_qd': 'R$3.600\nm² R$138',
  'quarto_suite_vaga': '[<li>1 Quarto</li>, <li>1 Suíte</li>, <li>1 Vaga</li>]'},
 {'id': '364429',
  'titulo': 'SGAN 912, ASA NORTE, BRASILIA',
  'preco_e_metro_qd': 'R$1.050\nm² R$38',
  'quarto_suite_vaga': '[<li>1 Quarto</li>, <li>1 Vaga</li>]'},
 {'id': '84159',
  'titulo': 'CCSW 2, SUDOESTE, BRASILIA',
  'preco_e_metro_qd': 'R$1.800\nm² R$50',
  'quarto_suite_vaga': '[<li>1 Quarto</li>, <li>1 Vaga</li>]'},
 {'id': '352275',
  'titulo': 'CLN 107, ASA NORTE, BRASILIA',
  'preco_e_metro_qd': 'R$980\nm² R$30',
  'quarto_suite_vaga': '[<li>1 Quarto</li>]'},
 {'id

##### Iremos agora transformar todos os dados em uma Data Frame

In [21]:
todos_os_imoveis = pd.DataFrame(columns=uni.keys())
todos_os_imoveis

,id,titulo,preco_e_metro_qd,quarto_suite_vaga


In [22]:
todos_os_imoveis = todos_os_imoveis.append(fichas)
todos_os_imoveis.head()

,id,titulo,preco_e_metro_qd,quarto_suite_vaga
0,361049,"SEPS 713/913, ASA SUL, BRASILIA",R$1.200\nm² R$42,"[<li>1 Quarto</li>, <li>1 Vaga</li>]"
1,359071,"SCRN 708/709, ASA NORTE, BRASILIA",R$1.200\nm² R$40,[<li>1 Quarto</li>]
2,429066,"CCSW 1, SUDOESTE, BRASILIA",R$3.600\nm² R$138,"[<li>1 Quarto</li>, <li>1 Suíte</li>, <li>1 Va..."
3,364429,"SGAN 912, ASA NORTE, BRASILIA",R$1.050\nm² R$38,"[<li>1 Quarto</li>, <li>1 Vaga</li>]"
4,84159,"CCSW 2, SUDOESTE, BRASILIA",R$1.800\nm² R$50,"[<li>1 Quarto</li>, <li>1 Vaga</li>]"


### Limpeza e Preparação de Dados

##### Os dados foram extraídos, porém ainda é necessário seja realizada a limpeza e preparação deles, a fim de que seja possível analisá-los.

In [23]:
todos_os_imoveis[['endereco', 'bairro', 'cidade']] = todos_os_imoveis['titulo'].str.split(',', 2, expand=True)
todos_os_imoveis.tail()

,id,titulo,preco_e_metro_qd,quarto_suite_vaga,endereco,bairro,cidade
973,331522,"QMSW 5, SUDOESTE, BRASILIA",R$1.200\nm² R$50,"[<li>1 Quarto</li>, <li>1 Vaga</li>]",QMSW 5,SUDOESTE,BRASILIA
974,431519,"Quadra 301 Rua A Conjunto 2, NORTE, AGUAS CLARAS",R$950\nm² R$31,"[<li>1 Quarto</li>, <li>1 Vaga</li>]",Quadra 301 Rua A Conjunto 2,NORTE,AGUAS CLARAS
975,322680,"SCRN 708/709 Bloco D, ASA NORTE, BRASILIA",R$1.250\nm² R$43,"[<li>1 Quarto</li>, <li>1 Suíte</li>, <li>1 Va...",SCRN 708/709 Bloco D,ASA NORTE,BRASILIA
976,431088,"CLSW 104 Bloco B, SUDOESTE, BRASILIA",R$900\nm² R$25,[<li>1 Quarto</li>],CLSW 104 Bloco B,SUDOESTE,BRASILIA
977,286149,"CLN 306, ASA NORTE, BRASILIA",R$750\nm² R$27,[<li>1 Quarto</li>],CLN 306,ASA NORTE,BRASILIA


##### Começando pela tabela de preços e metros quadrado, pode-se perceber que essas duas informações estão na mesma coluna do Data Frame, sendo assim, é necessário separar tais informações e limpar os caracteres desnecessários.

In [24]:
todos_os_imoveis['preco_e_metro_qd'] = todos_os_imoveis['preco_e_metro_qd'].str.strip('R$')
todos_os_imoveis[['preco','metro_quadrado(m²)']] = todos_os_imoveis['preco_e_metro_qd'].str.split('\n',1, expand=True)
todos_os_imoveis.head()

,id,titulo,preco_e_metro_qd,quarto_suite_vaga,endereco,bairro,cidade,preco,metro_quadrado(m²)
0,361049,"SEPS 713/913, ASA SUL, BRASILIA",1.200\nm² R$42,"[<li>1 Quarto</li>, <li>1 Vaga</li>]",SEPS 713/913,ASA SUL,BRASILIA,1.200,m² R$42
1,359071,"SCRN 708/709, ASA NORTE, BRASILIA",1.200\nm² R$40,[<li>1 Quarto</li>],SCRN 708/709,ASA NORTE,BRASILIA,1.200,m² R$40
2,429066,"CCSW 1, SUDOESTE, BRASILIA",3.600\nm² R$138,"[<li>1 Quarto</li>, <li>1 Suíte</li>, <li>1 Va...",CCSW 1,SUDOESTE,BRASILIA,3.600,m² R$138
3,364429,"SGAN 912, ASA NORTE, BRASILIA",1.050\nm² R$38,"[<li>1 Quarto</li>, <li>1 Vaga</li>]",SGAN 912,ASA NORTE,BRASILIA,1.050,m² R$38
4,84159,"CCSW 2, SUDOESTE, BRASILIA",1.800\nm² R$50,"[<li>1 Quarto</li>, <li>1 Vaga</li>]",CCSW 2,SUDOESTE,BRASILIA,1.800,m² R$50


In [25]:
todos_os_imoveis['metro_quadrado(m²)'] = todos_os_imoveis['metro_quadrado(m²)'].str.strip("m² R$ \AcPermuta")
todos_os_imoveis['metro_quadrado(m²)']

0       42
1       40
2      138
3       38
4       50
      ... 
973     50
974     31
975     43
976     25
977     27
Name: metro_quadrado(m²), Length: 978, dtype: object

In [26]:
todos_os_imoveis.head()

,id,titulo,preco_e_metro_qd,quarto_suite_vaga,endereco,bairro,cidade,preco,metro_quadrado(m²)
0,361049,"SEPS 713/913, ASA SUL, BRASILIA",1.200\nm² R$42,"[<li>1 Quarto</li>, <li>1 Vaga</li>]",SEPS 713/913,ASA SUL,BRASILIA,1.200,42
1,359071,"SCRN 708/709, ASA NORTE, BRASILIA",1.200\nm² R$40,[<li>1 Quarto</li>],SCRN 708/709,ASA NORTE,BRASILIA,1.200,40
2,429066,"CCSW 1, SUDOESTE, BRASILIA",3.600\nm² R$138,"[<li>1 Quarto</li>, <li>1 Suíte</li>, <li>1 Va...",CCSW 1,SUDOESTE,BRASILIA,3.600,138
3,364429,"SGAN 912, ASA NORTE, BRASILIA",1.050\nm² R$38,"[<li>1 Quarto</li>, <li>1 Vaga</li>]",SGAN 912,ASA NORTE,BRASILIA,1.050,38
4,84159,"CCSW 2, SUDOESTE, BRASILIA",1.800\nm² R$50,"[<li>1 Quarto</li>, <li>1 Vaga</li>]",CCSW 2,SUDOESTE,BRASILIA,1.800,50


##### Com as duas colunas 'preco' e 'metro_quadrado(m²)' definidas, podemos excluir a sua coluna de origem, a 'preco_e_metro_qd'

In [27]:
todos_os_imoveis.drop(['preco_e_metro_qd','titulo'], axis=1, inplace=True)
todos_os_imoveis

,id,quarto_suite_vaga,endereco,bairro,cidade,preco,metro_quadrado(m²)
0,361049,"[<li>1 Quarto</li>, <li>1 Vaga</li>]",SEPS 713/913,ASA SUL,BRASILIA,1.200,42
1,359071,[<li>1 Quarto</li>],SCRN 708/709,ASA NORTE,BRASILIA,1.200,40
2,429066,"[<li>1 Quarto</li>, <li>1 Suíte</li>, <li>1 Va...",CCSW 1,SUDOESTE,BRASILIA,3.600,138
3,364429,"[<li>1 Quarto</li>, <li>1 Vaga</li>]",SGAN 912,ASA NORTE,BRASILIA,1.050,38
4,84159,"[<li>1 Quarto</li>, <li>1 Vaga</li>]",CCSW 2,SUDOESTE,BRASILIA,1.800,50
...,...,...,...,...,...,...,...
973,331522,"[<li>1 Quarto</li>, <li>1 Vaga</li>]",QMSW 5,SUDOESTE,BRASILIA,1.200,50
974,431519,"[<li>1 Quarto</li>, <li>1 Vaga</li>]",Quadra 301 Rua A Conjunto 2,NORTE,AGUAS CLARAS,950,31
975,322680,"[<li>1 Quarto</li>, <li>1 Suíte</li>, <li>1 Va...",SCRN 708/709 Bloco D,ASA NORTE,BRASILIA,1.250,43
976,431088,[<li>1 Quarto</li>],CLSW 104 Bloco B,SUDOESTE,BRASILIA,900,25


##### O próximo passo é separar a coluna 'quarto_suite_vaga' de maneira que cada uma dessas informações se transforme em uma coluna.
##### Vamos começar realizando a limpeza dos caratéres desnecessários.

In [28]:
todos_os_imoveis['quarto_suite_vaga'] = todos_os_imoveis['quarto_suite_vaga'].str.replace('<li>','')
todos_os_imoveis['quarto_suite_vaga'] = todos_os_imoveis['quarto_suite_vaga'].str.replace('</li>','')
todos_os_imoveis['quarto_suite_vaga'] = todos_os_imoveis['quarto_suite_vaga'].str.strip('[]')
todos_os_imoveis['quarto_suite_vaga']

0               1 Quarto, 1 Vaga
1                       1 Quarto
2      1 Quarto, 1 Suíte, 1 Vaga
3               1 Quarto, 1 Vaga
4               1 Quarto, 1 Vaga
                 ...            
973             1 Quarto, 1 Vaga
974             1 Quarto, 1 Vaga
975    1 Quarto, 1 Suíte, 1 Vaga
976                     1 Quarto
977                     1 Quarto
Name: quarto_suite_vaga, Length: 978, dtype: object

In [29]:
todos_os_imoveis['quarto_suite_vaga'].value_counts()

1 Quarto                      478
1 Quarto, 1 Vaga              317
1 Quarto, 1 Suíte, 1 Vaga      94
1 Quarto, 1 Suíte              48
                               21
1 Vaga                          7
1 Suíte, 1 Vaga                 3
1 Quarto, 1 Suíte, 2 Vagas      3
1 Suíte                         2
1 Quarto, 2 Vagas               2
2 Quartos                       2
6 Quartos                       1
Name: quarto_suite_vaga, dtype: int64

##### Note que, ao realizar a contagem dos valores resultantes da nossa limpeza de dados, temos alguns problemas: existem anúncios que não apresentaram nenhuma informação sobre a quantidade de quartos, suítes ou vagas; quando não se tem uma informação como a existencia de uma suíte, esse dado simplesmente não aparece; e existem informações que estão no singular e outras que estão no plural (Ex: 'Quarto' e 'Quartos'). Sendo assim, a limpeza dos dados necessitará de alguams etapas a mais a fim de ter como resultado final 3 colunas: n° de quartos, n° de suítes e n° de vagas.

##### Vamos começar então pelo n° de quartos:

teste = todos_os_imoveis['quarto_suite_vaga'].str.split(',',2,expand=True)
teste[0] = teste[0].str.replace("Quartos","Quarto")
teste[0] = teste[0].str.split('Quarto',1,expand=True)
todos_os_imoveis['quarto_suite_vaga'] = todos_os_imoveis['quarto_suite_vaga'].str.replace('Quartos','Quarto')
lista = []
for i in todos_os_imoveis['quarto_suite_vaga']:
    if ("Quarto" in i) == True:
        lista.append(i)
    else:
        lista.append(np.nan)
lista

In [30]:
todos_os_imoveis['quarto_suite_vaga'] = todos_os_imoveis['quarto_suite_vaga'].str.replace('Quartos','Quarto')
lista = []
for i in todos_os_imoveis['quarto_suite_vaga']:
    if ("Quarto" in i) == True:
        lista.append(i)
    else:
        lista.append(np.nan)
lista

['1 Quarto, 1 Vaga',
 '1 Quarto',
 '1 Quarto, 1 Suíte, 1 Vaga',
 '1 Quarto, 1 Vaga',
 '1 Quarto, 1 Vaga',
 '1 Quarto',
 '1 Quarto, 1 Vaga',
 '1 Quarto',
 '1 Quarto, 1 Vaga',
 '1 Quarto, 1 Vaga',
 '1 Quarto',
 '1 Quarto',
 '1 Quarto, 1 Vaga',
 '1 Quarto, 1 Suíte, 1 Vaga',
 '1 Quarto',
 '1 Quarto, 1 Vaga',
 '1 Quarto, 1 Suíte, 1 Vaga',
 '1 Quarto',
 '1 Quarto, 1 Vaga',
 '1 Quarto',
 '1 Quarto',
 '1 Quarto',
 '1 Quarto',
 '1 Quarto, 1 Vaga',
 '1 Quarto, 1 Suíte, 1 Vaga',
 '1 Quarto, 1 Vaga',
 '1 Quarto',
 '1 Quarto',
 '1 Quarto, 1 Suíte, 1 Vaga',
 '1 Quarto',
 '1 Quarto, 1 Suíte, 1 Vaga',
 '1 Quarto, 1 Suíte',
 '1 Quarto',
 '1 Quarto, 1 Vaga',
 '1 Quarto, 1 Suíte, 1 Vaga',
 '1 Quarto',
 '1 Quarto',
 '1 Quarto',
 '1 Quarto, 1 Suíte',
 '1 Quarto, 1 Suíte, 1 Vaga',
 '1 Quarto, 1 Vaga',
 '1 Quarto, 1 Vaga',
 '1 Quarto, 1 Vaga',
 '1 Quarto',
 '1 Quarto, 1 Vaga',
 '1 Quarto, 1 Suíte, 1 Vaga',
 '1 Quarto, 1 Vaga',
 '1 Quarto, 1 Suíte',
 '1 Quarto',
 '1 Quarto, 1 Suíte',
 '1 Quarto',
 '1 Quarto',

##### No código acima, foi criada uma lista e incluído todos os anúncios que continham 'Quarto' ou 'Quartos' na sua descrição. Os anúncios que não possuiam nenhuma dessas duas palavras receberam o valor de 'NaN'.

##### Agora, iremos transformar essa lista em um DataFrame e separar as informações que queremos através do método .split

In [31]:
coluna_com_quarto = {'quartos_prep': lista}
coluna_com_quarto = pd.DataFrame(coluna_com_quarto)
coluna_com_quarto[['n° de quartos','outras_info']] = coluna_com_quarto['quartos_prep'].str.split('Quarto',1,expand=True)
coluna_com_quarto

,quartos_prep,n° de quartos,outras_info
0,"1 Quarto, 1 Vaga",1,", 1 Vaga"
1,1 Quarto,1,
2,"1 Quarto, 1 Suíte, 1 Vaga",1,", 1 Suíte, 1 Vaga"
3,"1 Quarto, 1 Vaga",1,", 1 Vaga"
4,"1 Quarto, 1 Vaga",1,", 1 Vaga"
...,...,...,...
973,"1 Quarto, 1 Vaga",1,", 1 Vaga"
974,"1 Quarto, 1 Vaga",1,", 1 Vaga"
975,"1 Quarto, 1 Suíte, 1 Vaga",1,", 1 Suíte, 1 Vaga"
976,1 Quarto,1,


##### Veja bem, a coluna 'n° de quartos' já está pronta, agora resta incluí-la no nosso DataFrame principal.

In [32]:
incluindo_num_de_quartos = pd.concat([todos_os_imoveis, coluna_com_quarto], axis=1)
incluindo_num_de_quartos.drop(['quartos_prep','outras_info'], axis=1, inplace=True)
todos_os_imoveis_com_quarto = incluindo_num_de_quartos
todos_os_imoveis_com_quarto

,id,quarto_suite_vaga,endereco,bairro,cidade,preco,metro_quadrado(m²),n° de quartos
0,361049,"1 Quarto, 1 Vaga",SEPS 713/913,ASA SUL,BRASILIA,1.200,42,1
1,359071,1 Quarto,SCRN 708/709,ASA NORTE,BRASILIA,1.200,40,1
2,429066,"1 Quarto, 1 Suíte, 1 Vaga",CCSW 1,SUDOESTE,BRASILIA,3.600,138,1
3,364429,"1 Quarto, 1 Vaga",SGAN 912,ASA NORTE,BRASILIA,1.050,38,1
4,84159,"1 Quarto, 1 Vaga",CCSW 2,SUDOESTE,BRASILIA,1.800,50,1
...,...,...,...,...,...,...,...,...
973,331522,"1 Quarto, 1 Vaga",QMSW 5,SUDOESTE,BRASILIA,1.200,50,1
974,431519,"1 Quarto, 1 Vaga",Quadra 301 Rua A Conjunto 2,NORTE,AGUAS CLARAS,950,31,1
975,322680,"1 Quarto, 1 Suíte, 1 Vaga",SCRN 708/709 Bloco D,ASA NORTE,BRASILIA,1.250,43,1
976,431088,1 Quarto,CLSW 104 Bloco B,SUDOESTE,BRASILIA,900,25,1


lista2 = []
for i in todos_os_imoveis['quarto_suite_vaga']:
    if ("Quarto" in i) == True:
        lista2.append(i)
    else:
        lista2.append(np.nan)

##### Agora que incluímos o quarto, vamos fazer o mesmo processo com as suítes:

In [33]:
todos_os_imoveis_com_quarto['quarto_suite_vaga'].value_counts()

1 Quarto                      478
1 Quarto, 1 Vaga              317
1 Quarto, 1 Suíte, 1 Vaga      94
1 Quarto, 1 Suíte              48
                               21
1 Vaga                          7
1 Suíte, 1 Vaga                 3
1 Quarto, 1 Suíte, 2 Vagas      3
2 Quarto                        2
1 Suíte                         2
1 Quarto, 2 Vagas               2
6 Quarto                        1
Name: quarto_suite_vaga, dtype: int64

In [34]:
lista2 = []
for i in todos_os_imoveis_com_quarto['quarto_suite_vaga']:
    if ("Suíte" in i) == True:
        lista2.append(i)
    else:
        lista2.append(np.nan)
lista2

[nan,
 nan,
 '1 Quarto, 1 Suíte, 1 Vaga',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 '1 Quarto, 1 Suíte, 1 Vaga',
 nan,
 nan,
 '1 Quarto, 1 Suíte, 1 Vaga',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 '1 Quarto, 1 Suíte, 1 Vaga',
 nan,
 nan,
 nan,
 '1 Quarto, 1 Suíte, 1 Vaga',
 nan,
 '1 Quarto, 1 Suíte, 1 Vaga',
 '1 Quarto, 1 Suíte',
 nan,
 nan,
 '1 Quarto, 1 Suíte, 1 Vaga',
 nan,
 nan,
 nan,
 '1 Quarto, 1 Suíte',
 '1 Quarto, 1 Suíte, 1 Vaga',
 nan,
 nan,
 nan,
 nan,
 nan,
 '1 Quarto, 1 Suíte, 1 Vaga',
 nan,
 '1 Quarto, 1 Suíte',
 nan,
 '1 Quarto, 1 Suíte',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 '1 Quarto, 1 Suíte, 1 Vaga',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 '1 Quarto, 1 Suíte, 1 Vaga',
 nan,
 nan,
 nan,
 nan,
 nan,
 '1 Suíte, 1 Vaga',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan

##### Transformando a 'lista2' em um DataFrame:

In [35]:
coluna_com_suite = {'suites_prep': lista2}
coluna_com_suite = pd.DataFrame(coluna_com_suite)
coluna_com_suite

,suites_prep
0,NaN
1,NaN
2,"1 Quarto, 1 Suíte, 1 Vaga"
3,NaN
4,NaN
...,...
973,NaN
974,NaN
975,"1 Quarto, 1 Suíte, 1 Vaga"
976,NaN


In [36]:
coluna_com_suite[['n° de suite','outras_info']] = coluna_com_suite['suites_prep'].str.split('Suíte',1,expand=True)
coluna_com_suite

,suites_prep,n° de suite,outras_info
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,"1 Quarto, 1 Suíte, 1 Vaga","1 Quarto, 1",", 1 Vaga"
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
973,NaN,NaN,NaN
974,NaN,NaN,NaN
975,"1 Quarto, 1 Suíte, 1 Vaga","1 Quarto, 1",", 1 Vaga"
976,NaN,NaN,NaN


##### Note que, após realizar a separação das linhas que contém a palavra 'suíte' ainda é necessário limpar os dados para que resultem apenas os números.

In [37]:
coluna_com_suite['n° de suite'] = coluna_com_suite['n° de suite'].str.replace("1 Quarto","")
coluna_com_suite['n° de suite'].value_counts()

, 1     145
1         5
Name: n° de suite, dtype: int64

In [38]:
coluna_com_suite['n° de suite'] = coluna_com_suite['n° de suite'].str.strip(', ')
coluna_com_suite['n° de suite'].value_counts()

1    150
Name: n° de suite, dtype: int64

In [39]:
coluna_com_suite

,suites_prep,n° de suite,outras_info
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,"1 Quarto, 1 Suíte, 1 Vaga",1,", 1 Vaga"
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
973,NaN,NaN,NaN
974,NaN,NaN,NaN
975,"1 Quarto, 1 Suíte, 1 Vaga",1,", 1 Vaga"
976,NaN,NaN,NaN


##### Resta apenas incluír a coluna 'n° de suites' no nosso DataFrame principal

In [40]:
incluindo_num_de_suites = pd.concat([todos_os_imoveis_com_quarto, coluna_com_suite], axis=1)
incluindo_num_de_suites.drop(['suites_prep','outras_info'], axis=1, inplace=True)
todos_os_imoveis_com_quarto_e_suite = incluindo_num_de_suites
todos_os_imoveis_com_quarto_e_suite

,id,quarto_suite_vaga,endereco,bairro,cidade,preco,metro_quadrado(m²),n° de quartos,n° de suite
0,361049,"1 Quarto, 1 Vaga",SEPS 713/913,ASA SUL,BRASILIA,1.200,42,1,NaN
1,359071,1 Quarto,SCRN 708/709,ASA NORTE,BRASILIA,1.200,40,1,NaN
2,429066,"1 Quarto, 1 Suíte, 1 Vaga",CCSW 1,SUDOESTE,BRASILIA,3.600,138,1,1
3,364429,"1 Quarto, 1 Vaga",SGAN 912,ASA NORTE,BRASILIA,1.050,38,1,NaN
4,84159,"1 Quarto, 1 Vaga",CCSW 2,SUDOESTE,BRASILIA,1.800,50,1,NaN
...,...,...,...,...,...,...,...,...,...
973,331522,"1 Quarto, 1 Vaga",QMSW 5,SUDOESTE,BRASILIA,1.200,50,1,NaN
974,431519,"1 Quarto, 1 Vaga",Quadra 301 Rua A Conjunto 2,NORTE,AGUAS CLARAS,950,31,1,NaN
975,322680,"1 Quarto, 1 Suíte, 1 Vaga",SCRN 708/709 Bloco D,ASA NORTE,BRASILIA,1.250,43,1,1
976,431088,1 Quarto,CLSW 104 Bloco B,SUDOESTE,BRASILIA,900,25,1,NaN


##### Por fim, vamos fazer o mesmo processo para obter a coluna 'n° de vagas':

In [41]:
todos_os_imoveis_com_quarto_e_suite['quarto_suite_vaga'].value_counts()

1 Quarto                      478
1 Quarto, 1 Vaga              317
1 Quarto, 1 Suíte, 1 Vaga      94
1 Quarto, 1 Suíte              48
                               21
1 Vaga                          7
1 Suíte, 1 Vaga                 3
1 Quarto, 1 Suíte, 2 Vagas      3
2 Quarto                        2
1 Suíte                         2
1 Quarto, 2 Vagas               2
6 Quarto                        1
Name: quarto_suite_vaga, dtype: int64

In [42]:
todos_os_imoveis_com_quarto_e_suite

,id,quarto_suite_vaga,endereco,bairro,cidade,preco,metro_quadrado(m²),n° de quartos,n° de suite
0,361049,"1 Quarto, 1 Vaga",SEPS 713/913,ASA SUL,BRASILIA,1.200,42,1,NaN
1,359071,1 Quarto,SCRN 708/709,ASA NORTE,BRASILIA,1.200,40,1,NaN
2,429066,"1 Quarto, 1 Suíte, 1 Vaga",CCSW 1,SUDOESTE,BRASILIA,3.600,138,1,1
3,364429,"1 Quarto, 1 Vaga",SGAN 912,ASA NORTE,BRASILIA,1.050,38,1,NaN
4,84159,"1 Quarto, 1 Vaga",CCSW 2,SUDOESTE,BRASILIA,1.800,50,1,NaN
...,...,...,...,...,...,...,...,...,...
973,331522,"1 Quarto, 1 Vaga",QMSW 5,SUDOESTE,BRASILIA,1.200,50,1,NaN
974,431519,"1 Quarto, 1 Vaga",Quadra 301 Rua A Conjunto 2,NORTE,AGUAS CLARAS,950,31,1,NaN
975,322680,"1 Quarto, 1 Suíte, 1 Vaga",SCRN 708/709 Bloco D,ASA NORTE,BRASILIA,1.250,43,1,1
976,431088,1 Quarto,CLSW 104 Bloco B,SUDOESTE,BRASILIA,900,25,1,NaN


In [43]:
todos_os_imoveis_com_quarto_e_suite['quarto_suite_vaga'] = todos_os_imoveis_com_quarto_e_suite['quarto_suite_vaga'].str.replace('Vagas','Vaga')
lista3 = []
for i in todos_os_imoveis_com_quarto_e_suite['quarto_suite_vaga']:
    if ("Vaga" in i) == True:
        lista3.append(i)
    else:
        lista3.append(np.nan)
lista3

['1 Quarto, 1 Vaga',
 nan,
 '1 Quarto, 1 Suíte, 1 Vaga',
 '1 Quarto, 1 Vaga',
 '1 Quarto, 1 Vaga',
 nan,
 '1 Quarto, 1 Vaga',
 nan,
 '1 Quarto, 1 Vaga',
 '1 Quarto, 1 Vaga',
 nan,
 nan,
 '1 Quarto, 1 Vaga',
 '1 Quarto, 1 Suíte, 1 Vaga',
 nan,
 '1 Quarto, 1 Vaga',
 '1 Quarto, 1 Suíte, 1 Vaga',
 nan,
 '1 Quarto, 1 Vaga',
 nan,
 nan,
 nan,
 nan,
 '1 Quarto, 1 Vaga',
 '1 Quarto, 1 Suíte, 1 Vaga',
 '1 Quarto, 1 Vaga',
 nan,
 nan,
 '1 Quarto, 1 Suíte, 1 Vaga',
 nan,
 '1 Quarto, 1 Suíte, 1 Vaga',
 nan,
 nan,
 '1 Quarto, 1 Vaga',
 '1 Quarto, 1 Suíte, 1 Vaga',
 nan,
 nan,
 nan,
 nan,
 '1 Quarto, 1 Suíte, 1 Vaga',
 '1 Quarto, 1 Vaga',
 '1 Quarto, 1 Vaga',
 '1 Quarto, 1 Vaga',
 nan,
 '1 Quarto, 1 Vaga',
 '1 Quarto, 1 Suíte, 1 Vaga',
 '1 Quarto, 1 Vaga',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 '1 Quarto, 1 Vaga',
 '1 Quarto, 1 Vaga',
 '1 Quarto, 1 Vaga',
 '1 Quarto, 1 Suíte, 1 Vaga',
 '1 Quarto, 1 Vaga',
 '1 Quarto, 1 Vaga',
 '1 Quarto, 1 Vaga',
 nan,
 nan,
 nan,
 '1 Quarto, 1 Vaga',
 '1 Quarto, 1 Va

##### Transformando a 'lista3' em um DataFrame

In [44]:
coluna_com_vaga = {'vaga_prep': lista3}
coluna_com_vaga = pd.DataFrame(coluna_com_vaga)
coluna_com_vaga

,vaga_prep
0,"1 Quarto, 1 Vaga"
1,NaN
2,"1 Quarto, 1 Suíte, 1 Vaga"
3,"1 Quarto, 1 Vaga"
4,"1 Quarto, 1 Vaga"
...,...
973,"1 Quarto, 1 Vaga"
974,"1 Quarto, 1 Vaga"
975,"1 Quarto, 1 Suíte, 1 Vaga"
976,NaN


In [45]:
coluna_com_vaga[['n° de vagas','outras_info']] = coluna_com_vaga['vaga_prep'].str.split('Vaga',1,expand=True)
coluna_com_vaga

,vaga_prep,n° de vagas,outras_info
0,"1 Quarto, 1 Vaga","1 Quarto, 1",
1,NaN,NaN,NaN
2,"1 Quarto, 1 Suíte, 1 Vaga","1 Quarto, 1 Suíte, 1",
3,"1 Quarto, 1 Vaga","1 Quarto, 1",
4,"1 Quarto, 1 Vaga","1 Quarto, 1",
...,...,...,...
973,"1 Quarto, 1 Vaga","1 Quarto, 1",
974,"1 Quarto, 1 Vaga","1 Quarto, 1",
975,"1 Quarto, 1 Suíte, 1 Vaga","1 Quarto, 1 Suíte, 1",
976,NaN,NaN,NaN


In [46]:
coluna_com_vaga['n° de vagas'].value_counts()

1 Quarto, 1              317
1 Quarto, 1 Suíte, 1      94
1                          7
1 Suíte, 1                 3
1 Quarto, 1 Suíte, 2       3
1 Quarto, 2                2
Name: n° de vagas, dtype: int64

In [47]:
coluna_com_vaga['n° de vagas'] = coluna_com_vaga['n° de vagas'].str.replace("1 Quarto, ","")
coluna_com_vaga['n° de vagas'] = coluna_com_vaga['n° de vagas'].str.replace("1 Suíte, ","")
coluna_com_vaga['n° de vagas'].value_counts()

1     421
2       5
Name: n° de vagas, dtype: int64

In [48]:
coluna_com_vaga

,vaga_prep,n° de vagas,outras_info
0,"1 Quarto, 1 Vaga",1,
1,NaN,NaN,NaN
2,"1 Quarto, 1 Suíte, 1 Vaga",1,
3,"1 Quarto, 1 Vaga",1,
4,"1 Quarto, 1 Vaga",1,
...,...,...,...
973,"1 Quarto, 1 Vaga",1,
974,"1 Quarto, 1 Vaga",1,
975,"1 Quarto, 1 Suíte, 1 Vaga",1,
976,NaN,NaN,NaN


##### Vamos incluír a coluna 'n° de vagas' no nosso DataFrame final:

In [62]:
incluindo_num_de_vagas = pd.concat([todos_os_imoveis_com_quarto_e_suite, coluna_com_vaga], axis=1)
incluindo_num_de_vagas.drop(['vaga_prep','outras_info'], axis=1, inplace=True)
todos_os_imoveis_completos = incluindo_num_de_vagas
todos_os_imoveis_completos

,id,quarto_suite_vaga,endereco,bairro,cidade,preco,metro_quadrado(m²),n° de quartos,n° de suite,n° de vagas
0,361049,"1 Quarto, 1 Vaga",SEPS 713/913,ASA SUL,BRASILIA,1.200,42,1,NaN,1
1,359071,1 Quarto,SCRN 708/709,ASA NORTE,BRASILIA,1.200,40,1,NaN,NaN
2,429066,"1 Quarto, 1 Suíte, 1 Vaga",CCSW 1,SUDOESTE,BRASILIA,3.600,138,1,1,1
3,364429,"1 Quarto, 1 Vaga",SGAN 912,ASA NORTE,BRASILIA,1.050,38,1,NaN,1
4,84159,"1 Quarto, 1 Vaga",CCSW 2,SUDOESTE,BRASILIA,1.800,50,1,NaN,1
...,...,...,...,...,...,...,...,...,...,...
973,331522,"1 Quarto, 1 Vaga",QMSW 5,SUDOESTE,BRASILIA,1.200,50,1,NaN,1
974,431519,"1 Quarto, 1 Vaga",Quadra 301 Rua A Conjunto 2,NORTE,AGUAS CLARAS,950,31,1,NaN,1
975,322680,"1 Quarto, 1 Suíte, 1 Vaga",SCRN 708/709 Bloco D,ASA NORTE,BRASILIA,1.250,43,1,1,1
976,431088,1 Quarto,CLSW 104 Bloco B,SUDOESTE,BRASILIA,900,25,1,NaN,NaN


##### A coluna 'quarto_suite_vaga' não é mais necessária, então vamos excluí-la

In [63]:
todos_os_imoveis_completos.drop(['quarto_suite_vaga'], axis=1, inplace=True)
todos_os_imoveis_completos

,id,endereco,bairro,cidade,preco,metro_quadrado(m²),n° de quartos,n° de suite,n° de vagas
0,361049,SEPS 713/913,ASA SUL,BRASILIA,1.200,42,1,NaN,1
1,359071,SCRN 708/709,ASA NORTE,BRASILIA,1.200,40,1,NaN,NaN
2,429066,CCSW 1,SUDOESTE,BRASILIA,3.600,138,1,1,1
3,364429,SGAN 912,ASA NORTE,BRASILIA,1.050,38,1,NaN,1
4,84159,CCSW 2,SUDOESTE,BRASILIA,1.800,50,1,NaN,1
...,...,...,...,...,...,...,...,...,...
973,331522,QMSW 5,SUDOESTE,BRASILIA,1.200,50,1,NaN,1
974,431519,Quadra 301 Rua A Conjunto 2,NORTE,AGUAS CLARAS,950,31,1,NaN,1
975,322680,SCRN 708/709 Bloco D,ASA NORTE,BRASILIA,1.250,43,1,1,1
976,431088,CLSW 104 Bloco B,SUDOESTE,BRASILIA,900,25,1,NaN,NaN


##### Por fim, vamos realizar os ajustes finais para poder começar a análise dos dados.

In [64]:
todos_os_imoveis_completos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 978 entries, 0 to 977
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  978 non-null    object
 1   endereco            978 non-null    object
 2   bairro              978 non-null    object
 3   cidade              978 non-null    object
 4   preco               978 non-null    object
 5   metro_quadrado(m²)  978 non-null    object
 6   n° de quartos       945 non-null    object
 7   n° de suite         150 non-null    object
 8   n° de vagas         426 non-null    object
dtypes: object(9)
memory usage: 76.4+ KB


In [65]:
todos_os_imoveis_completos['preco'] = todos_os_imoveis_completos['preco'].str.replace('.','')
todos_os_imoveis_completos['metro_quadrado(m²)'] = todos_os_imoveis_completos['metro_quadrado(m²)'].str.replace('.','')


In [71]:
todos_os_imoveis_completos[['n° de quartos', 'n° de suite','n° de vagas']] = todos_os_imoveis_completos[['n° de quartos', 'n° de suite','n° de vagas']].fillna(0)
todos_os_imoveis_completos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 978 entries, 0 to 977
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  978 non-null    object 
 1   endereco            978 non-null    object 
 2   bairro              978 non-null    object 
 3   cidade              978 non-null    object 
 4   preco               978 non-null    int64  
 5   metro_quadrado(m²)  978 non-null    int64  
 6   n° de quartos       978 non-null    float64
 7   n° de suite         978 non-null    float64
 8   n° de vagas         978 non-null    float64
dtypes: float64(3), int64(2), object(4)
memory usage: 76.4+ KB


In [72]:
todos_os_imoveis_completos[['preco','metro_quadrado(m²)','n° de quartos', 'n° de suite','n° de vagas']] = todos_os_imoveis_completos[['preco','metro_quadrado(m²)','n° de quartos', 'n° de suite','n° de vagas']].apply(pd.to_numeric)

In [73]:
todos_os_imoveis_completos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 978 entries, 0 to 977
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  978 non-null    object 
 1   endereco            978 non-null    object 
 2   bairro              978 non-null    object 
 3   cidade              978 non-null    object 
 4   preco               978 non-null    int64  
 5   metro_quadrado(m²)  978 non-null    int64  
 6   n° de quartos       978 non-null    float64
 7   n° de suite         978 non-null    float64
 8   n° de vagas         978 non-null    float64
dtypes: float64(3), int64(2), object(4)
memory usage: 76.4+ KB
